In [ ]:
# Copyright 2024 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# 使用 AutoSxS 範例評估大型語言模型

<table align="left">
  <td style="text-align: center">
    <a href="https://colab.research.google.com/github/doggy8088/generative-ai/blob/main/gemini/evaluation/evaluate_gemini_with_autosxs.zh.ipynb">
      <img width="32px" src="https://www.gstatic.com/pantheon/images/bigquery/welcome_page/colab-logo.svg" alt="Google Colaboratory 標誌"><br> 在 Colab 中執行
    </a>
  </td>
  <td style="text-align: center">
    <a href="https://console.cloud.google.com/vertex-ai/colab/import/https:%2F%2Fraw.githubusercontent.com%2Fdoggy8088%2Fgenerative-ai%2Fmain%2Fgemini%2Fevaluation%2Fevaluate_gemini_with_autosxs.zh.ipynb">
      <img width="32px" src="https://lh3.googleusercontent.com/JmcxdQi-qOpctIvWKgPtrzZdJJK-J3sWE1RsfjZNwshCFgE_9fULcNpuXYTilIR2hjwN" alt="Google Cloud Colab Enterprise 標誌"><br> 在 Colab Enterprise 中執行
    </a>
  </td>
  <td style="text-align: center">
    <a href="https://github.com/doggy8088/generative-ai/blob/main/gemini/evaluation/evaluate_gemini_with_autosxs.zh.ipynb">
      <img width="32px" src="https://upload.wikimedia.org/wikipedia/commons/9/91/Octicons-mark-github.svg" alt="GitHub 標誌"><br> 在 GitHub 上查看
    </a>
  </td>
  <td style="text-align: center">
    <a href="https://console.cloud.google.com/vertex-ai/workbench/deploy-notebook?download_url=https://raw.githubusercontent.com/doggy8088/generative-ai/main/gemini/evaluation/evaluate_gemini_with_autosxs.zh.ipynb">
      <img src="https://www.gstatic.com/images/branding/gcpiconscolors/vertexai/v1/32px.svg" alt="Vertex AI 標誌"><br> 在 Vertex AI Workbench 中開啟
    </a>
  </td>    
</table>


| | |
|-|-|
|作者 | [Ivan Nardini](https://github.com/inardini) |


**_注意_** : 已在下列環境中測試過這個筆記本：

* Python 版本 = 3.9


## 概述

Vertex AI Model Evaluation AutoSxS 是一種 LLM 評估工具，使用戶能夠比較 Google 第一類型和第三方 LLM 的效能。

作為預覽版本的其中一環，AutoSxS 僅支援根據一些預先定義的準則比較用於「摘要」和「問答」作業的模型。請查看 [文件](https://cloud.google.com/vertex-ai/docs/generative-ai/models/side-by-side-eval#autosxs) 以深入了解。

本教學展示如何使用 Vertex AI Model Evaluation AutoSxS 來比較模型並在摘要作業中檢查人工比對。


### 目標

在本教學課程中，你會學習如何使用 Vertex AI Model Evaluation AutoSxS 在摘要任務中比較兩個 LLM 預測 (其中一個模型為 Gemini 1.0 Pro)。

本教學課程使用下列 Google Cloud ML 服務及資源：

- Vertex AI Model Evaluation

執行步驟包括：

- 閱讀評量資料
- 定義 AutoSxS 模型評量管道
- 執行評量管道作業
- 檢查判斷、彙總指標與人為比對


### 資料集

本資料集為 [XSum](https://huggingface.co/datasets/EdinburghNLP/xsum) 資料集的修改樣本，用於評估抽象式單篇文件摘要系統。


### 成本

本教學使用下列 Google Cloud 的計費元件：

* Vertex AI
* Cloud Storage

進一步了解 [Vertex AI 定價](https://cloud.google.com/vertex-ai/pricing) 及 [Cloud Storage 定價](https://cloud.google.com/storage/pricing)，並使用 [定價計算器](https://cloud.google.com/products/calculator/)，根據你預計的使用量估算成本。


## 安裝

安裝執行此筆記本所需的以下套件。


In [ ]:
! pip3 install --user --upgrade --quiet google-cloud-aiplatform

### 僅 Colab：取消以下Cell註解，重新啟動核心。


In [ ]:
# import IPython

# app = IPython.Application.instance()
# app.kernel.do_shutdown(True)

## 開始之前

### 設定 Google Cloud 專案

**下列步驟是必要的，不論你的筆記本環境是什麼。** 

1. [選擇或建立 Google Cloud 專案](https://console.cloud.google.com/cloud-resource-manager)。當你第一次建立帳戶時，你會取得 300 美元的免費額度，可以抵用你的運算/儲存成本。

2. [確定已為你的專案啟用帳務處理](https://cloud.google.com/billing/docs/how-to/modify-project)。

3. [啟用 Vertex AI API](https://console.cloud.google.com/flows/enableapi?apiid=aiplatform.googleapis.com)。

4.如果你在本地端執行這個筆記本，你需要安裝 [Cloud SDK](https://cloud.google.com/sdk)。


#### 設定你的專案 ID

**如果你不知道自己的專案 ID** , 請嘗試以下動作：
* 執行 `gcloud config list`。
* 執行 `gcloud projects list`。
* 參閱支援頁面：[尋找專案 ID](https://support.google.com/googleapi/answer/7014113)


In [ ]:
PROJECT_ID = "your-project-id"  # @param {type:"string"}

# Set the project id
! gcloud config set project {PROJECT_ID}

#### 區域

你也可以變更 Vertex AI 使用的 `REGION` 變數。進一步了解 [Vertex AI 地區](https://cloud.google.com/vertex-ai/docs/general/locations)。


In [ ]:
REGION = "us-central1"  # @param {type: "string"}

### UUID

我們定義一個 UUID 產生函式，以避免在筆記本中建立的資源上產生資源名稱衝突。


In [ ]:
import random
import string


def generate_uuid(length: int = 8) -> str:
    """Generate a uuid of a specifed length (default=8)."""
    return "".join(random.choices(string.ascii_lowercase + string.digits, k=length))


UUID = generate_uuid()

### 驗證你的 Google Cloud 帳戶

視你的 Jupyter 環境而定，你可能必須手動驗證。請按照以下相關說明進行操作。


**1. Vertex AI Workbench** 
* 不做任何事情，因為你已驗證身份。


**2. Local JupyterLab 執行個體 (移除註解並執行)：** 


In [ ]:
# ! gcloud auth login

**3. Colab, 取消註釋並執行：** 


In [ ]:
# from google.colab import auth
# auth.authenticate_user()

**4. 服務帳戶或其他** 
* 詳見如何於 https://cloud.google.com/storage/docs/gsutil/commands/iam#ch-examples 上授予服務帳戶 Cloud Storage 權限。


### 建立 Cloud Storage 儲存空間

建立一個儲存空間儲存暫時產品，例如資料集。


In [ ]:
BUCKET_URI = f"gs://autosxs-demo-{UUID}"  # @param {type:"string"}

**只有你的儲存空間不存在時，才會建立儲存空間：** 執行下列Cell來建立你的 Cloud Storage 儲存空間。


In [ ]:
! gsutil mb -l {REGION} -p {PROJECT_ID} {BUCKET_URI}

### 匯入函式庫


In [ ]:
import os
import pprint

import pandas as pd
from google.cloud import aiplatform
from google.protobuf.json_format import MessageToDict
from IPython.display import HTML, display

### 定義常數


In [ ]:
EVALUATION_FILE_URI = (
    "gs://github-repo/evaluate-gemini/sum_eval_gemini_dataset_001.jsonl"
)
HUMAN_EVALUATION_FILE_URI = (
    "gs://github-repo/evaluate-gemini/sum_human_eval_gemini_dataset_001.jsonl"
)
TEMPLATE_URI = "https://us-kfp.pkg.dev/ml-pipeline/llm-rlhf/autosxs-template/2.8.0"
PIPELINE_ROOT = f"{BUCKET_URI}/pipeline"

### 幫手


In [ ]:
def print_autosxs_judgments(df, n=3):
    """Print AutoSxS judgments in the notebook"""

    style = "white-space: pre-wrap; width: 800px; overflow-x: auto;"
    df = df.sample(n=n)

    for index, row in df.iterrows():
        if row["confidence"] >= 0.5:
            display(
                HTML(
                    f"<h2>Document:</h2> <div style='{style}'>{row['id_columns']['document']}</div>"
                )
            )
            display(
                HTML(
                    f"<h2>Response A:</h2> <div style='{style}'>{row['response_a']}</div>"
                )
            )
            display(
                HTML(
                    f"<h2>Response B:</h2> <div style='{style}'>{row['response_b']}</div>"
                )
            )
            display(
                HTML(
                    f"<h2>Explanation:</h2> <div style='{style}'>{row['explanation']}</div>"
                )
            )
            display(
                HTML(
                    f"<h2>Confidence score:</h2> <div style='{style}'>{row['confidence']}</div>"
                )
            )
            display(HTML("<hr>"))


def print_aggregated_metrics(scores):
    """Print AutoSxS aggregated metrics"""

    score_b = round(win_rate_metrics["autosxs_model_b_win_rate"] * 100)
    display(
        HTML(
            f"<h3>AutoSxS Autorater prefers {score_b}% of time Model B over Model A </h3>"
        )
    )


def print_human_preference_metrics(metrics):
    """Print AutoSxS Human-preference alignment metrics"""
    display(
        HTML(
            f"<h3>AutoSxS Autorater prefers {score_b}% of time Model B over Model A </h3>"
        )
    )

### 初始化 Python 版的 Vertex AI SDK

為你的專案初始化 Python 版的 Vertex AI SDK。


In [ ]:
aiplatform.init(project=PROJECT_ID, location=REGION, staging_bucket=BUCKET_URI)

## 使用 Vertex AI Model Evaluation AutoSxS 評估 LLM

假設你已在摘要任務中獲得你生成式 LLM 預測。若要使用 AutoSXS 在 Vertex AI 上評估 LLM (例如於 Vertex AI 上的 Gemini 1.0 Pro) 與其他模型的表現，你需要遵循下列步驟進行評估：

1.   **準備評估資料集** ：收集評估所需的提示、脈絡、已生成回應及人類偏好。

2.   **轉換評估資料集** ：將資料集轉換成 JSONL 格式並儲存在 Cloud Storage 儲存空間中。或者，你可以將資料集儲存到 BigQuery 資料表中。

3.   **執行模型評估工作** ：使用 Vertex AI 執行模型評估工作，以評估 LLM 的效能。


### 閱讀評量資料

在此摘要使用案例中，你使用 `sum_eval_gemini_dataset_001`，一種 JSONL 格式的評量資料集，包含沒有人類偏好的內容回應配對。

在資料集中，每一列代表單一範例。該資料集包含身分識別欄位，例如「id」和「文件」，這些欄位用於識別每個獨特範例。「文件」欄位包含新聞文章，將予以摘要。

儘管該資料集沒有提示和內容的 [資料欄位](https://cloud.google.com/vertex-ai/docs/generative-ai/models/side-by-side-eval#prep-eval-dataset)，但它包含預先產生的預測。這些預測包含根據 LLM 任務產生的回應，其中「response_a」和「response_b」代表不同的文章摘要。

**注意：** 實驗時，你只能提供幾個範例。文件建議至少 400 個範例，以確保高品質的整體指標。


In [ ]:
evaluation_gemini_df = pd.read_json(EVALUATION_FILE_URI, lines=True)
evaluation_gemini_df.head()

### 執行模型評量作業

AutoSxS 依賴 Vertex AI 管線執行模型評量。這裡可看到一些必要的管線參數：

*  `evaluation_dataset` 指出評量資料集的位置。這裡是 JSONL Cloud Storage URI。

*   `id_columns` 用於辨別唯一的評量範例。如你所料，這裡有 `id` 和 `document` 欄位。

*   `task` 指出你要評量的任務類型，格式為 `{task}@{version}`。可以是 `summarization` 或 `question_answer`。這裡有 `summarization`。

*   `autorater_prompt_parameters` 設定自動評分器任務行為。你可以指定推理指令來引導任務完成，並設定在任務執行期間要參考的推理脈絡。

最後，你必須提供 `response_column_a` 和 `response_column_b`，其中包含預定義預測的欄位名稱，以計算評量指標。這裡分別為 `response_a` 和 `response_b`。

若要深入了解所有支援的參數，請參閱 [官方文件](https://cloud.google.com/vertex-ai/docs/generative-ai/models/side-by-side-eval#perform-eval)。


In [ ]:
display_name = f"autosxs-eval-{generate_uuid()}"
parameters = {
    "evaluation_dataset": EVALUATION_FILE_URI,
    "id_columns": ["id", "document"],
    "task": "summarization",
    "autorater_prompt_parameters": {
        "inference_context": {"column": "document"},
        "inference_instruction": {"template": "Summarize the following text: "},
    },
    "response_column_a": "response_a",
    "response_column_b": "response_b",
}

在你定義模型評估參數後，你可以使用 Vertex AI Python SDK 及預先定義的管道範本執行模型評估管道作業。


In [ ]:
job = aiplatform.PipelineJob(
    job_id=display_name,
    display_name=display_name,
    pipeline_root=os.path.join(BUCKET_URI, display_name),
    template_path=TEMPLATE_URI,
    parameter_values=parameters,
    enable_caching=False,
)
job.run(sync=True)

### 評估結果

評估管線成功執行後，你可以透過檢視 Vertex AI Pipelines UI 中的管線本身所產生的成果，以及使用 Vertex AI Python SDK 在筆記本環境中產生的成果，來檢閱評估結果。

AutoSXS 產生三種類型的評估結果：判斷表、總計量測標準和對齊量測標準 (如果提供了人為偏好)。


### AutoSxS 判斷

判斷表格包含衡量指標，提供每個範例中 LLM 效能的見解。

對於每一組回應，判斷表格包含一個 `choice` 欄位，說明根據自動評分員使用的評分基準，哪一個回應較佳。

每個選擇都有介於 0 到 1 之間的 `信心得分` 欄位，代表自動評分員對評分的信心等級。

最後但同樣重要的是，AutoSXS 提供自動評分員偏好一方回應勝過另一方的理由說明。

以下是 AutoSxS 判斷輸出的範例。


In [ ]:
for details in job.task_details:
    if details.task_name == "autosxs-arbiter":
        break

judgments_uri = MessageToDict(details.outputs["judgments"]._pb)["artifacts"][0]["uri"]
judgments_df = pd.read_json(judgments_uri, lines=True)

In [ ]:
print_autosxs_judgments(judgments_df)

### AutoSxS 聚合指標

AutoSxS 也提供聚合指標作為額外的評估結果。這些勝率指標是利用判斷表格計算出的，用來決定 AutoRater 偏好每種模型回應的百分比。

這些指標有助於快速找出在所評估任務中哪一種模型較佳。

以下是 AutoSxS 聚合指標的範例。


In [ ]:
for details in job.task_details:
    if details.task_name == "autosxs-metrics-computer":
        break

win_rate_metrics = MessageToDict(details.outputs["autosxs_metrics"]._pb)["artifacts"][
    0
]["metadata"]
print_aggregated_metrics(win_rate_metrics)

### 人類偏好一致性指標

在查看你的初步 AutoSxS 評估結果後，你可能會想知道 Autorater 評估與人類評量者的觀點之間的一致性。

AutoSxS 支援人類偏好以驗證 Autorater 評估。

若要檢查與人類偏好資料集的一致性，你需要將基本事實新增為一欄到 `evaluation_dataset`，並將該欄位名稱傳遞給 `human_preference_column`。


#### 閱讀評估資料

關於評估資料集，在此案例中 `sum_human_eval_gemini_dataset_001` 資料集也包含人類偏好。

以下是資料集的範例。


In [ ]:
human_evaluation_gemini_df = pd.read_json(HUMAN_EVALUATION_FILE_URI, lines=True)
human_evaluation_gemini_df.head()

#### 執行模型評估作業

對於 AutoSXS 處理程序，你必須在處理程序參數中指定人為偏好欄位。

然後，你可以使用下述 Vertex AI Python SDK 執行評估處理程序作業。


In [ ]:
display_name = f"autosxs-human-eval-{generate_uuid()}"
parameters = {
    "evaluation_dataset": HUMAN_EVALUATION_FILE_URI,
    "id_columns": ["id", "document"],
    "task": "summarization",
    "autorater_prompt_parameters": {
        "inference_context": {"column": "document"},
        "inference_instruction": {"template": "Summarize the following text: "},
    },
    "response_column_a": "response_a",
    "response_column_b": "response_b",
    "human_preference_column": "actual",
}

In [ ]:
job = aiplatform.PipelineJob(
    job_id=display_name,
    display_name=display_name,
    pipeline_root=os.path.join(BUCKET_URI, display_name),
    template_path=TEMPLATE_URI,
    parameter_values=parameters,
    enable_caching=False,
)
job.run(sync=True)

### 取得與人類偏好一致的彙總測量指標

與之前的彙總測量指標相比，現在管道回傳使用所提供的偏好資料後所取得的其他測量資料。

除了準確度、準確性及召回等眾所周知的測量指標外，你將會收到人類偏好分數及自動評分器偏好分數。這些分數表示評量之間的同意程度。並為了簡化比較，提供Cohen's Kappa。Cohen's Kappa測量自動評分器與人類評分器之間的同意程度。它的範圍從0到1，其中0表示與隨機選擇相當的同意，1則表示完全同意。

以下是與人類偏好一致的彙總測量結果：


In [ ]:
for details in job.task_details:
    if details.task_name == "autosxs-metrics-computer":
        break

human_aligned_metrics = {
    k: round(v, 3)
    for k, v in MessageToDict(details.outputs["autosxs_metrics"]._pb)["artifacts"][0][
        "metadata"
    ].items()
}
pprint.pprint(human_aligned_metrics)

## 清理

如要清理此專案中使用的所有 Google Cloud 資源，你可以 [刪除用於教學課程的 Google Cloud 專案](https://cloud.google.com/resource-manager/docs/creating-managing-projects#shutting_down_projects)。

否則，你可以刪除在教學課程中建立的個別資源。


In [ ]:
import os

# Delete Model Evaluation pipeline run
delete_pipeline = True
if delete_pipeline or os.getenv("IS_TESTING"):
    job.delete()

# Delete Cloud Storage objects that were created
delete_bucket = True
if delete_bucket or os.getenv("IS_TESTING"):
    ! gsutil -m rm -r $BUCKET_URI